In [ ]:
import numpy as np

def objective(x):
    """目标函数：例如最小化 x[0]^2 + x[1]^2"""
    return x[0]**2 + x[1]**2

def constraint(x):
    """约束条件：例如 g(x) = x[0] + x[1] - 1，g(x) >= 0"""
    return x[0] + x[1] - 1

def linear_approximation(func, xk, perturbation=1e-5):
    """线性化函数，通过对目标函数和约束函数进行泰勒展开，估算其梯度"""
    n = len(xk)
    approx = np.zeros(n)
    for i in range(n):
        xk_perturbed = xk.copy()
        xk_perturbed[i] += perturbation
        approx[i] = (func(xk_perturbed) - func(xk)) / perturbation
    return approx

def cobyla(objective, constraint, x0, max_iter=100, tol=1e-6, rho=1.0):
    """COBYLA算法"""
    xk = np.array(x0)  # 当前点
    n = len(x0)
    m = 1  # 只有一个约束g(x) >= 0
    for iteration in range(max_iter):
        # 1. 计算目标和约束的线性化
        grad_f = linear_approximation(objective, xk)  # 目标函数的梯度
        grad_g = linear_approximation(constraint, xk)  # 约束函数的梯度
        f_xk = objective(xk)
        g_xk = constraint(xk)
        
        # 2. 构建单纯形
        simplex = np.array([xk + np.random.randn(n) * rho for _ in range(n+1)])
        
        # 3. 线性拟合：目标函数和约束的近似
        a0 = grad_f
        b0 = f_xk
        ag = grad_g
        bg = g_xk
        
        # 4. 解线性优化子问题
        # 在信赖域内最小化线性近似模型，目标是最小化 a0^T d
        # 线性约束为 ag^T d + bg >= 0, 并且 ||d|| <= rho
        d = np.linalg.solve(np.outer(a0, a0) + np.outer(ag, ag), -a0)
        
        # 5. 更新点和信赖域
        xk_next = xk + d
        if objective(xk_next) < f_xk:
            xk = xk_next
            rho *= 1.2  # 扩大信赖域
        else:
            rho *= 0.5  # 缩小信赖域
        
        # 6. 收敛判断
        if np.linalg.norm(d) < tol:
            print(f"Converged at iteration {iteration}")
            break

    return xk

# 初始点
x0 = [0.5, 0.5]
result = cobyla(objective, constraint, x0)
print(f"Optimization result: {result}")
